In [1]:
import numpy as np
import pandas as pd
from bert_serving.client import BertClient
from keras.utils import np_utils
from keras.models import Sequential
from keras.optimizers import Adam,SGD
from keras.layers import Dense, Input, Dropout, LSTM, Activation, Convolution1D, MaxPooling1D, Flatten, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
train = pd.read_csv('./data/training data/labeled_training_set.csv')
test = pd.read_csv('./data/test data/unlabeled_test_set.csv')

In [41]:
train.head()

,sentenceid,author,nchar,created_utc,score,subreddit,label,full_text,wordcount,id,Emotional_disclosure,Information_disclosure,Support,General_support,Info_support,Emo_support
0,36.0,LeniaLilac,135,1532518450,81,offmychest,husband boyfriend,Get two nice notebooks and write it down for e...,12,91px39,0,0,0,0,0,0
1,47.0,call-me-mama-t,115,1533351628,59,offmychest,husband boyfriend,I<U+0092>m sobbing reading this thread!,5,91px39,1,1,0,0,0,0
2,57.0,BluebirdOnAHill,46,1532513532,35,offmychest,husband boyfriend,Hope you have a nice day,6,91px39,0,0,1,1,0,0
3,58.0,MenudoMenudo,157,1532519594,34,offmychest,husband boyfriend,My wife came in when I was around half way thr...,33,91px39,1,1,0,0,0,0
4,74.0,modistabeat,191,1533722813,30,offmychest,husband boyfriend,I am crying a lot of happy tears right now.,10,91px39,1,0,0,0,0,0


In [42]:
test.head()

,sentenceid,author,nchar,created_utc,score,subreddit,label,full_text,wordcount,id
0,66026,blargghonkk,357,1491057453,2,offmychest,husband wife,Wouldn't want the baby to fall apart on the hi...,10,62hoet
1,96956,Spikekuji,100,1531173984,1,offmychest,wife,You<U+0092>d be in good company there.,6,8xa92s
2,238262,maximize123,279,1501968080,1,CasualConversation,husband,"What a cliche, but as a 20 year old in college...",22,6ru5kk
3,381299,kilawl,2353,1507209034,1,CasualConversation,boyfriend,"Well, when I thought I lost my best friend for...",14,74alb3
4,30238,Munkyed,914,1533323463,3,offmychest,husband wife,I love my brother and all twelve of his kids.,10,944icl


In [34]:
np.unique(train['Emotional_disclosure'])

array([0, 1])

In [35]:
train.head()

,sentenceid,author,nchar,created_utc,score,subreddit,label,full_text,wordcount,id,Emotional_disclosure,Information_disclosure,Support,General_support,Info_support,Emo_support
0,36.0,LeniaLilac,135,1532518450,81,offmychest,husband boyfriend,Get two nice notebooks and write it down for e...,12,91px39,0,0,0,0,0,0
1,47.0,call-me-mama-t,115,1533351628,59,offmychest,husband boyfriend,I<U+0092>m sobbing reading this thread!,5,91px39,1,1,0,0,0,0
2,57.0,BluebirdOnAHill,46,1532513532,35,offmychest,husband boyfriend,Hope you have a nice day,6,91px39,0,0,1,1,0,0
3,58.0,MenudoMenudo,157,1532519594,34,offmychest,husband boyfriend,My wife came in when I was around half way thr...,33,91px39,1,1,0,0,0,0
4,74.0,modistabeat,191,1533722813,30,offmychest,husband boyfriend,I am crying a lot of happy tears right now.,10,91px39,1,0,0,0,0,0


In [45]:
test.head()

,sentenceid,author,nchar,created_utc,score,subreddit,label,full_text,wordcount,id
0,66026,blargghonkk,357,1491057453,2,offmychest,husband wife,Wouldn't want the baby to fall apart on the hi...,10,62hoet
1,96956,Spikekuji,100,1531173984,1,offmychest,wife,You<U+0092>d be in good company there.,6,8xa92s
2,238262,maximize123,279,1501968080,1,CasualConversation,husband,"What a cliche, but as a 20 year old in college...",22,6ru5kk
3,381299,kilawl,2353,1507209034,1,CasualConversation,boyfriend,"Well, when I thought I lost my best friend for...",14,74alb3
4,30238,Munkyed,914,1533323463,3,offmychest,husband wife,I love my brother and all twelve of his kids.,10,944icl


In [46]:
from bert_serving.client import BertClient

In [ ]:
bc = BertClient()

In [3]:
sent_train = []
for ix in train['full_text']:
    sent_train.append(ix)

In [38]:
print(sent_train[1])

I<U+0092>m sobbing reading this thread!


In [45]:
print(len(sent_train))
#sent_train=np.array(sent_train)
#print(sent_train.shape)

12860


In [48]:
enc_train = bc.encode(sent_train)

In [49]:
print(enc_train.shape)

(12860, 768)


In [15]:
np.save('./enc_train.npy',enc_train)

In [16]:
sent_test = []
for ix in test['full_text']:
    sent_test.append(ix)

In [17]:
print(len(sent_test))

5000


In [18]:
enc_test = bc.encode(sent_test)

In [19]:
print(enc_test.shape)

(5000, 768)


In [20]:
np.save('./enc_test.npy',enc_test)

In [4]:
enc_train = np.load('./enc_train.npy')

In [5]:
Y_train = train.values[:,15]

In [6]:
Y_train=np.array(Y_train)

In [7]:
Y_train = np_utils.to_categorical(Y_train)

In [71]:
print(Y_train)

[[1. 0.]
 [0. 1.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [7]:
# Y_test = np_utils.to_categorical(Y_test)

In [8]:
from keras.layers import Bidirectional,Flatten
import keras

In [9]:
from keras_self_attention import SeqSelfAttention

In [10]:
from keras.layers import Bidirectional,Flatten

In [11]:
from sklearn.model_selection import RepeatedKFold, cross_val_score
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import StratifiedKFold,KFold


In [12]:
indices=np.arange(12860)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test ,indices_train,indices_test= train_test_split(enc_train, Y_train,indices, test_size=0.1,stratify=Y_train)

In [13]:
model = Sequential()
model.add(Convolution1D(64,(3),activation='relu', input_shape=(50,768)))
model.add(Convolution1D(128,(3),activation='relu'))
#model.add(Convolution1D(128,(1),activatio\n='relu'))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
#model.add(LSTM(128, return_sequences=False))
#model.add(Dropout(0.5))
model.add(SeqSelfAttention(
    attention_width=15,
    attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
    attention_activation=None,
    kernel_regularizer=keras.regularizers.l2(1e-6),
    use_attention_bias=False,
    name='Attention',
))
model.add(Flatten())
model.add(Dense(1000))
model.add(Dropout(0.5))
model.add(Dense(64))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))
model.summary()
model.compile(loss='categorical_crossentropy',optimizer=SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True),metrics=['accuracy'])
 


W1205 09:34:34.853469 140535258793792 deprecation_wrapper.py:119] From /media/data_dump_2/Mehak/biocv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1205 09:34:34.868797 140535258793792 deprecation_wrapper.py:119] From /media/data_dump_2/Mehak/biocv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1205 09:34:34.870572 140535258793792 deprecation_wrapper.py:119] From /media/data_dump_2/Mehak/biocv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1205 09:34:35.877701 140535258793792 deprecation_wrapper.py:119] From /media/data_dump_2/Mehak/biocv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please u

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 48, 64)            147520    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 46, 128)           24704     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 46, 256)           263168    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 46, 512)           1050624   
_________________________________________________________________
Attention (SeqSelfAttention) (None, 46, 512)           262144    
_________________________________________________________________
flatten_1 (Flatten)          (None, 23552)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              23553000  
__________

In [14]:

kfold=KFold(n_splits=5)
print(kfold)

KFold(n_splits=5, random_state=None, shuffle=False)


In [15]:
early_stopping = EarlyStopping(min_delta = 0.001, mode = 'max', monitor='val_acc', patience = 5)
callback = [early_stopping]

In [16]:
checkpoint=ModelCheckpoint('model_bert_final_cv_cat_6_final.h5',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

In [17]:

score=[]
for train, test in kfold.split(X_train, y_train):
    model.fit(X_train[train],y_train[train],validation_data=(X_train[test],y_train[test]),
                epochs = 20, batch_size=16,callbacks=[checkpoint,early_stopping])
    score.append(model.evaluate(X_test,y_test,verbose=0))

W1205 09:34:43.535734 140535258793792 deprecation.py:323] From /media/data_dump_2/Mehak/biocv/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 9259 samples, validate on 2315 samples
Epoch 1/20
9259/9259 [==============================] - 132s 14ms/step - loss: 0.3037 - acc: 0.9210 - val_loss: 0.2816 - val_acc: 0.9192
Epoch 2/20
9259/9259 [==============================] - 127s 14ms/step - loss: 0.2562 - acc: 0.9225 - val_loss: 0.2491 - val_acc: 0.9192
Epoch 4/20
9259/9259 [==============================] - 127s 14ms/step - loss: 0.2369 - acc: 0.9225 - val_loss: 0.2309 - val_acc: 0.9218
Epoch 5/20
9259/9259 [==============================] - 129s 14ms/step - loss: 0.2225 - acc: 0.9225 - val_loss: 0.2212 - val_acc: 0.9240
Epoch 6/20
9259/9259 [==============================] - 127s 14ms/step - loss: 0.2088 - acc: 0.9256 - val_loss: 0.2154 - val_acc: 0.9253
Epoch 7/20
9259/9259 [==============================] - 128s 14ms/step - loss: 0.2011 - acc: 0.9285 - val_loss: 0.2127 - val_acc: 0.9253
Epoch 8/20
4640/9259 [==============>...............] - ETA: 58s - loss: 0.2020 - acc: 0.9310

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



9259/9259 [==============================] - 129s 14ms/step - loss: 0.1716 - acc: 0.9382 - val_loss: 0.1749 - val_acc: 0.9387
Train on 9259 samples, validate on 2315 samples
Epoch 1/20
9259/9259 [==============================] - 131s 14ms/step - loss: 0.1791 - acc: 0.9335 - val_loss: 0.1454 - val_acc: 0.9499
Epoch 2/20
9259/9259 [==============================] - 131s 14ms/step - loss: 0.1755 - acc: 0.9357 - val_loss: 0.1463 - val_acc: 0.9421
Epoch 3/20
9259/9259 [==============================] - 130s 14ms/step - loss: 0.1749 - acc: 0.9356 - val_loss: 0.1412 - val_acc: 0.9477
Epoch 4/20
9259/9259 [==============================] - 132s 14ms/step - loss: 0.1737 - acc: 0.9362 - val_loss: 0.1436 - val_acc: 0.9425
Epoch 5/20
9259/9259 [==============================] - 130s 14ms/step - loss: 0.1698 - acc: 0.9388 - val_loss: 0.1410 - val_acc: 0.9477
Epoch 6/20
 896/9259 [=>............................] - ETA: 1:46 - loss: 0.1742 - acc: 0.9286

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [18]:
#model2 = build_model()
#model2.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001),metrics=['accuracy'])


# load 3rd epoch weights
model.load_weights('./model_bert_final_cv_cat_6_final.h5')
y_pred=model.predict(X_test)
scores2 = model.evaluate(X_test,y_test,verbose=0)

In [19]:
print(scores2[1]*100)

92.30171073094868


In [20]:
Y_pred=[]
for i in range(len(y_pred)):
    Y_pred.append(np.argmax(y_pred[i]))



Y_test=[]
for i in range(len(y_test)):
    Y_test.append(np.argmax(y_test[i]))
    
    
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score

print('Accuracy:', accuracy_score(Y_test, Y_pred))
print('F1 score:', f1_score(Y_test, Y_pred))
print('Recall:', recall_score(Y_test, Y_pred))
print ('Precision:', precision_score(Y_test, Y_pred))
print('\n clasification report:\n', classification_report(Y_test, Y_pred))
print('\n confussion matrix:\n',confusion_matrix(Y_test, Y_pred))



Accuracy: 0.9230171073094868
F1 score: 0.39999999999999997
Recall: 0.32673267326732675
Precision: 0.515625

 clasification report:
               precision    recall  f1-score   support

           0       0.94      0.97      0.96      1185
           1       0.52      0.33      0.40       101

    accuracy                           0.92      1286
   macro avg       0.73      0.65      0.68      1286
weighted avg       0.91      0.92      0.91      1286


 confussion matrix:
 [[1154   31]
 [  68   33]]


In [21]:
sentences_test=[]
for i in range(len(indices_test)):
    sentences_test.append(sent_train[indices_test[i]])
    

In [22]:
print(len(sentences_test))

1286


In [23]:
with open("fn_cv_cat_6.txt", "a") as f:    
    for i in range(len(X_test)):
        if np.argmax(y_pred[i])==0 and Y_test[i] == 0:
            print('Sentence', sentences_test[i], file=f)
            print('GT',Y_test[i],file=f)
            print('Pred',np.argmax(y_pred[i]),file=f)
        

In [24]:
with open("tp_cv_cat_6.txt", "a") as f:    
    for i in range(len(X_test)):
        if np.argmax(y_pred[i])==1 and Y_test[i] == 1:
            print('Sentence', sentences_test[i], file=f)
            print('GT',Y_test[i],file=f)
            print('Pred',np.argmax(y_pred[i]),file=f)

In [25]:
with open("fp_cv_cat_6.txt", "a") as f:    
    for i in range(len(X_test)):
        if np.argmax(y_pred[i])==1 and Y_test[i] == 0:
            print('Sentence',sentences_test[i], file=f)
            print('GT',Y_test[i],file=f)
            print('Pred',np.argmax(y_pred[i]),file=f)

In [26]:
with open("tn_cv_cat_6.txt", "a") as f:    
    for i in range(len(X_test)):
        if np.argmax(y_pred[i])==0 and Y_test[i] == 1:
            print('Sentence', sentences_test[i], file=f)
            print('GT',Y_test[i],file=f)
            print('Pred',np.argmax(y_pred[i]),file=f)

In [27]:
np.save("indices_test_cat_6",indices_test)

np.save("indices_train_cat_6",indices_train)

np.save("test_Set_cat_6",X_test)